In [2]:
from PIL import Image
import requests
from io import BytesIO
import json
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from skimage.filters import threshold_otsu
from skimage.io import imread, imsave
from skimage.color import rgb2gray
from skimage.measure import label, regionprops
import random
import math
import imageio
import imgaug as ia
from imgaug import augmenters as iaa
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage
import glob
import random
import shutil

In [3]:
labels = os.path.join('Pf','pf_01.json')
count = 11794
degs = [45,90,135,180,225,270,315]
with open(labels,'r') as json_file:
    data = json.load(json_file)
    images = data["_via_img_metadata"]
    for key in images:
        url = images[key]["filename"]
        leng = len(images[key]["regions"])
        bboxs = ""
        if leng > 0:
            name = "{:010d}".format(count)
            print("{} {}".format(count,url))
            response = requests.get(url)
            img = Image.open(BytesIO(response.content))
            w, h = img.size
            img.save(os.path.join('Pf','images',name+".jpg"))
            f = open(os.path.join('Pf','labels',name+".txt"),'w')
            rects = []
            for box in images[key]["regions"]:
                b = box["shape_attributes"]
                x1 = b["x"]/w
                y1 = b["y"]/h
                bw = b["width"]/w
                bh = b["height"]/h
                xc = x1 + (bw/2)
                yc = y1 + (bh/2)
                lbl = box["region_attributes"]["type"]
                if lbl == "1":
                    lbl = "2"
                    bboxs += "\n"+ lbl + " " + str(xc) + " " + str(yc) + " " + str(bw) + " " + str(bh)
                    rects.append(BoundingBox(x1=b["x"],y1=b["y"],x2=(b["x"] + b["width"]),y2=(b["y"] + b["height"]),label=lbl))
            bboxs = bboxs[1:]
            f.write(bboxs)
            f.close()
            count += 1
            img = np.array(img)
            bbs = BoundingBoxesOnImage(rects,shape=img.shape)
            for deg in degs:
                name = "{:010d}".format(count)
                image_aug, bbs_aug = iaa.Affine(rotate=deg,mode="reflect")(image=img, bounding_boxes=bbs)
                aug_h, aug_w, aug_c = image_aug.shape
                clip_off_bbs = bbs_aug.remove_out_of_image(fully=True,partly=True).clip_out_of_image()
                bboxs = ""
                for bb in clip_off_bbs.bounding_boxes:
                    x1 = bb.x1
                    y1 = bb.y1
                    x2 = bb.x2
                    y2 = bb.y2
                    bb_w = (x2-x1)/aug_w
                    bb_h = (y2-y1)/aug_h
                    bb_xc = (x1/aug_w) + (bb_w/2)
                    bb_yc = (y1/aug_h) + (bb_h/2)
                    bboxs += "\n"+ str(bb.label) + " " + str(bb_xc) + " " + str(bb_yc) + " " + str(bb_w) + " " + str(bb_h)
                bboxs = bboxs[1:]
                im_aug = Image.fromarray(image_aug)
                im_aug.save(os.path.join('Pf','images',name+".jpg"))
                f = open(os.path.join('Pf','labels',name+".txt"),'w')
                f.write(bboxs)
                f.close()
                print("{} {}".format(count,deg))
                count += 1

11794 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i01j17.jpg
11795 45
11796 90
11797 135
11798 180
11799 225
11800 270
11801 315
11802 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i01j21.jpg
11803 45
11804 90
11805 135
11806 180
11807 225
11808 270
11809 315
11810 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i02j15.jpg
11811 45
11812 90
11813 135
11814 180
11815 225
11816 270
11817 315
11818 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i02j18.jpg
11819 45
11820 90
11821 135
11822 180
11823 225
11824 270
11825 315
11826 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i02j21.jpg
11827 45
11828 90
11829 135
11830 180
11831 225
11832 270
11833 315
11834 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Defa

12155 45
12156 90
12157 135
12158 180
12159 225
12160 270
12161 315
12162 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i03j36.jpg
12163 45
12164 90
12165 135
12166 180
12167 225
12168 270
12169 315
12170 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i03j37.jpg
12171 45
12172 90
12173 135
12174 180
12175 225
12176 270
12177 315
12178 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i03j38.jpg
12179 45
12180 90
12181 135
12182 180
12183 225
12184 270
12185 315
12186 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i03j39.jpg
12187 45
12188 90
12189 135
12190 180
12191 225
12192 270
12193 315
12194 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i03j40.jpg
12195 45
12196 90
12197 135
12198 180
12199 225
12200 270
12201 315
12202 http://map.ku.ac

12519 225
12520 270
12521 315
12522 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i04j30.jpg
12523 45
12524 90
12525 135
12526 180
12527 225
12528 270
12529 315
12530 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i04j31.jpg
12531 45
12532 90
12533 135
12534 180
12535 225
12536 270
12537 315
12538 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i04j32.jpg
12539 45
12540 90
12541 135
12542 180
12543 225
12544 270
12545 315
12546 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i04j33.jpg
12547 45
12548 90
12549 135
12550 180
12551 225
12552 270
12553 315
12554 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i04j34.jpg
12555 45
12556 90
12557 135
12558 180
12559 225
12560 270
12561 315
12562 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/D

12883 45
12884 90
12885 135
12886 180
12887 225
12888 270
12889 315
12890 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i05j20.jpg
12891 45
12892 90
12893 135
12894 180
12895 225
12896 270
12897 315
12898 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i05j21.jpg
12899 45
12900 90
12901 135
12902 180
12903 225
12904 270
12905 315
12906 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i05j22.jpg
12907 45
12908 90
12909 135
12910 180
12911 225
12912 270
12913 315
12914 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i05j23.jpg
12915 45
12916 90
12917 135
12918 180
12919 225
12920 270
12921 315
12922 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i05j24.jpg
12923 45
12924 90
12925 135
12926 180
12927 225
12928 270
12929 315
12930 http://map.ku.ac

13247 225
13248 270
13249 315
13250 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i05j65.jpg
13251 45
13252 90
13253 135
13254 180
13255 225
13256 270
13257 315
13258 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i05j66.jpg
13259 45
13260 90
13261 135
13262 180
13263 225
13264 270
13265 315
13266 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i05j67.jpg
13267 45
13268 90
13269 135
13270 180
13271 225
13272 270
13273 315
13274 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i05j68.jpg
13275 45
13276 90
13277 135
13278 180
13279 225
13280 270
13281 315
13282 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i05j69.jpg
13283 45
13284 90
13285 135
13286 180
13287 225
13288 270
13289 315
13290 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/D

13611 45
13612 90
13613 135
13614 180
13615 225
13616 270
13617 315
13618 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i06j61.jpg
13619 45
13620 90
13621 135
13622 180
13623 225
13624 270
13625 315
13626 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i06j62.jpg
13627 45
13628 90
13629 135
13630 180
13631 225
13632 270
13633 315
13634 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i06j63.jpg
13635 45
13636 90
13637 135
13638 180
13639 225
13640 270
13641 315
13642 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i06j64.jpg
13643 45
13644 90
13645 135
13646 180
13647 225
13648 270
13649 315
13650 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i06j65.jpg
13651 45
13652 90
13653 135
13654 180
13655 225
13656 270
13657 315
13658 http://map.ku.ac

13975 225
13976 270
13977 315
13978 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i07j54.jpg
13979 45
13980 90
13981 135
13982 180
13983 225
13984 270
13985 315
13986 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i07j55.jpg
13987 45
13988 90
13989 135
13990 180
13991 225
13992 270
13993 315
13994 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i07j56.jpg
13995 45
13996 90
13997 135
13998 180
13999 225
14000 270
14001 315
14002 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i07j57.jpg
14003 45
14004 90
14005 135
14006 180
14007 225
14008 270
14009 315
14010 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i07j58.jpg
14011 45
14012 90
14013 135
14014 180
14015 225
14016 270
14017 315
14018 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/D

14339 45
14340 90
14341 135
14342 180
14343 225
14344 270
14345 315
14346 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i08j44.jpg
14347 45
14348 90
14349 135
14350 180
14351 225
14352 270
14353 315
14354 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i08j45.jpg
14355 45
14356 90
14357 135
14358 180
14359 225
14360 270
14361 315
14362 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i08j46.jpg
14363 45
14364 90
14365 135
14366 180
14367 225
14368 270
14369 315
14370 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i08j47.jpg
14371 45
14372 90
14373 135
14374 180
14375 225
14376 270
14377 315
14378 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i08j48.jpg
14379 45
14380 90
14381 135
14382 180
14383 225
14384 270
14385 315
14386 http://map.ku.ac

14702 180
14703 225
14704 270
14705 315
14706 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i09j36.jpg
14707 45
14708 90
14709 135
14710 180
14711 225
14712 270
14713 315
14714 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i09j37.jpg
14715 45
14716 90
14717 135
14718 180
14719 225
14720 270
14721 315
14722 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i09j38.jpg
14723 45
14724 90
14725 135
14726 180
14727 225
14728 270
14729 315
14730 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i09j39.jpg
14731 45
14732 90
14733 135
14734 180
14735 225
14736 270
14737 315
14738 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i09j40.jpg
14739 45
14740 90
14741 135
14742 180
14743 225
14744 270
14745 315
14746 http://map.ku.ac.th/imglabel/via/datasets/pf

15063 225
15064 270
15065 315
15066 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i10j21.jpg
15067 45
15068 90
15069 135
15070 180
15071 225
15072 270
15073 315
15074 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i10j22.jpg
15075 45
15076 90
15077 135
15078 180
15079 225
15080 270
15081 315
15082 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i10j23.jpg
15083 45
15084 90
15085 135
15086 180
15087 225
15088 270
15089 315
15090 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i10j24.jpg
15091 45
15092 90
15093 135
15094 180
15095 225
15096 270
15097 315
15098 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i10j25.jpg
15099 45
15100 90
15101 135
15102 180
15103 225
15104 270
15105 315
15106 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/D

In [6]:
# clear empty box
files = glob.glob("Pf/images/*")
count = 1
for file in files:
    img = file.split("/")[2]
    name = img.split('.')[0]
    f = open("Pf/labels/"+name+".txt","r")
    txt = f.readlines()
    if len(txt) == 0:
        print(img)
        os.remove("Pf/labels/"+name+".txt")
        os.remove(file)
        count += 1
print(count)

0000011803.jpg
0000011804.jpg
0000011805.jpg
0000011807.jpg
0000011808.jpg
0000011809.jpg
0000011819.jpg
0000011820.jpg
0000011823.jpg
0000011824.jpg
0000011957.jpg
0000011961.jpg
0000012003.jpg
0000012004.jpg
0000012005.jpg
0000012007.jpg
0000012008.jpg
0000012009.jpg
0000012011.jpg
0000012012.jpg
0000012013.jpg
0000012015.jpg
0000012016.jpg
0000012017.jpg
0000012019.jpg
0000012023.jpg
0000012235.jpg
0000012236.jpg
0000012239.jpg
0000012240.jpg
0000012299.jpg
0000012300.jpg
0000012301.jpg
0000012302.jpg
0000012303.jpg
0000012304.jpg
0000012305.jpg
0000012308.jpg
0000012312.jpg
0000012317.jpg
0000012321.jpg
0000012365.jpg
0000012369.jpg
0000012388.jpg
0000012389.jpg
0000012392.jpg
0000012393.jpg
0000012396.jpg
0000012397.jpg
0000012400.jpg
0000012401.jpg
0000012411.jpg
0000012412.jpg
0000012415.jpg
0000012416.jpg
0000012419.jpg
0000012420.jpg
0000012421.jpg
0000012423.jpg
0000012424.jpg
0000012425.jpg
0000012435.jpg
0000012436.jpg
0000012437.jpg
0000012439.jpg
0000012440.jpg
0000012441

In [ ]:
files = glob.glob("Pf/images/*.jpg")
random.shuffle(files)
NUM_TRAIN = round(len(files) * 0.8)
NUM_VALID = round(len(files) * 0.2)
print(len(files))
print(NUM_TRAIN)
print(NUM_VALID)
training_set = files[:NUM_TRAIN ]
valid_set = files[-NUM_VALID:]
test_set = valid_set[-120:]
valid_set = valid_set[:-120]

file1 = open(os.path.join("data","custom","train.txt"),"a+")
for line in training_set:
    file1.write("\n"+line)
file1.close()

file1 = open(os.path.join("data","custom","valid.txt"),"a+")
for line in valid_set:
    file1.write("\n"+line)
file1.close()

file1 = open(os.path.join("data","custom","test.txt"),"a+")
for line in test_set:
    file1.write("\n"+line)
file1.close()

In [ ]:
labels = os.path.join('Pf','pf_01.json')
count = 0
with open(labels,'r') as json_file:
    data = json.load(json_file)
    images = data["_via_img_metadata"]
    for key in images:
        url = images[key]["filename"]
        if count >= 120 and count <= 240:
            name = "{:010d}".format(count)
            print("{} {}".format(count,url))
            response = requests.get(url)
            img = Image.open(BytesIO(response.content))
            w, h = img.size
            img.save(os.path.join('data','samples',name+".jpg"))
        count += 1
    

In [ ]:
count = 9784
for i in range(count,12583):
    name = "{:010d}".format(i)
    print(name)
    os.remove(os.path.join("data/custom/images/",name+".jpg"))
    os.remove(os.path.join("data/custom/labels/",name+".txt"))